In [38]:
import pandas as pd
import random as rd
import pickle,glob,re

In [26]:
cumule=True

In [28]:
listeTirages=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage.pkl")

In [27]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [29]:
compositionLongitudinale={"1Ko":10,"10Ko":9,"100Ko":9,"1Mo":9}

In [44]:
serieLongitudinale=[]
for serie in compositionLongitudinale:
    for nom in rd.sample(tiragesNom(serie),compositionLongitudinale[serie]):
        m=re.match(ur"^.*-\d{6}-%s-(\d{2})-.*"%serie,nom)
        if m:
#            print m.group(1), nom
            serieLongitudinale.append((serie,nom))
        else:
            print "mauvais nom de fichier",nom
serieLongitudinale

mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-14-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-03-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-13-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-10-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-16-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-17-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-05-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-02-100Ko-Tirage.pkl
mauvais nom de fichier /Users/gilles/Box Sync/2015-Data/MGC-170330-07-100Ko-Tirage.pkl


[('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-19-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-24-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-05-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-15-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-12-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-40-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170330-1Ko-00-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-43-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-11-Tirage.pkl'),
 ('1Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-1Ko-49-Tirage.pkl'),
 ('10Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-10Ko-01-Tirage.pkl'),
 ('10Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-10Ko-02-Tirage.pkl'),
 ('10Ko', '/Users/gilles/Box Sync/2015-Data/MGC-170506-10Ko-13-Tirage.pkl'),
 ('10Ko', '/Users/g

In [8]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [19]:
cumulLexique=lireLexique(listeLexiques[0])
cumulLexique["tir1"]=0

In [20]:
for nTirage,tirage in enumerate(listeLexiques):
    lexique=lireLexique(tirage)
    if cumule:
        cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    else:
        cumulLexique["tir1"]=lexique["tir1"]
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)

(1000000,43468),
(2000000,52524),
(3000000,57454),
(4000000,60598),
(5000000,62706),
(6000000,64338),
(7000000,65629),
(8000000,66617),
(9000000,67378),
(10000000,67996),
(11000000,68494),
(12000000,68930),
(13000000,69318),
(14000000,69611),
(15000000,69874),
(16000000,70145),
(17000000,70349),
(18000000,70531),
(19000000,70718),
(20000000,70865),
(21000000,70997),
(22000000,71133),
(23000000,71249),
(24000000,71353),
(25000000,71446),
(26000000,71519),
(27000000,71617),
(28000000,71703),
(29000000,71779),
(30000000,71846),
(31000000,71900),
(32000000,71984),
(33000000,72038),
(34000000,72090),
(35000000,72136),
(36000000,72209),
(37000000,72260),
(38000000,72303),
(39000000,72348),
(40000000,72388),
(41000000,72431),
(42000000,72491),
(43000000,72551),
(44000000,72589),
(45000000,72631),
(46000000,72671),
(47000000,72717),
(48000000,72750),
(49000000,72782),
(50000000,72817),
(51000000,72847),
